**Rapport de Claire KOECHLIN et Eryne THOMAS - 31 mai 2023**
# Dynamical Model for Building Energy Management

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Sommaire
Partie 1 - Modèle
> Description du sujet  
Description des paramètres  
Calcul des Conductances Thermiques  
Calcul des Capacités Thermiques  
Equations Différentielles Algébriques

Partie 2 - Régime Permanent
> Système d'Equations Différentielles  
Représentation d'état

PARTIE 3 - Modèle Dynamique : Réponse à un échelon  
>Détermination du pas de temps  
Détermination du temps total  
Hypothèses de Simulation  
Résolution Numérique

## PARTIE 1 - Modèle #  
### Description du sujet #

![Scheme_building_Koechlin_Thomas.png](Scheme_building_Koechlin_Thomas.png)

Les portes et les murs sont en bois, $lambois$ est le coefficient de conductivité, $Csp$ est la capacité spécifique du bois et $mv$ la masse volumique.

In [ ]:
lambois = 0.077 
Csp = 1250
mv = 370 #kg/m3

$Up$ est le coefficient thermique de la porte, en bois, et $Uf$ est le coeffcient thermique de la fenêtre, prise en double vitrage.

In [ ]:
Up = 2.5
Uf = 1.5

On impose la température extérieure et On utilise un controleur dans chaque pièce pour imposer la température intérieure. Il y a également un flux dans chaque pièce lié à la présence de personnes.

![Thermal_circuit (1).jpeg](Thermal_circuit.jpeg)

### Description des paramètres #
$Text$ est la température à l'extérieur et $Tc$ est la température imposée à l'intérieur.  
$hconvext$ est le coefficent de convection sur la surface extérieure des murs.  $hconvint$ est le coefficient de convection sur la surface intérieure des murs.  
$Q1$ et $Q2$ correspond aux emissions de chaleur par les personnes et appareils présentes dans les pièces 1 et 2.

In [ ]:
Text = 5+273
Tc = 20+273
hconvext = 10 
hconvint = 2

Q1= 300 #1 personne = 80W
Q2= 300 
E = 100 #éclairement du soleil en W/m2
phi1 =E*Sm1*0.8 #corps gris
phi2 =E*Sm*0.8 #corps gris

$emext = 30 mm$ est l'épaisseur des murs en contact avec l'extérieur et $emint = 10 mm$ du mur entre les deux pièces.  
$ep = 10 mm$ est l'épaisseur des portes et $ef = 3 mm$ celle des fenêtres.  
$Spf = 2m²$ est la surface des portes et des fenêtres.  
$Sm1 = 26 m²$ est la surface du mur 1.  
$Sm = 28 m²$ est la surface du mur intérieur et du mur 2

In [ ]:
emext = 30*(10**(-3))
emint = 10*(10**(-3))
ep = 10*(10**(-3))
ef = 3*(10**(-3))
Spf = 2
Sm1 = 30-2*Spf
Sm = 30-Spf

Le volume d'air renouvellé par advection est $Va$, et $Vadot$ est le flux d'air.  $rhoair$ est la masse volumique de l'air et $Cair$ est la capacité thermique de l'air.

In [ ]:
Va = 10*10*3
ACH = 1
Va_dot = ACH/3600 * Va
rhoair = 1.3
Cair = 1

### Calcul des Conductances Thermiques #
#### Advection #
$G_{advection} = \rho_{air} . C_{air}. \dot{V_{a}}$

In [ ]:
Gad = rhoair*Cair*Va_dot

#### Convection #
$G_{convection} = h.S$  
$Gconvext1$ est la convection sur la surface extérieure du mur 1.  $Gconvint1$ est la convection sur la surface intérieure du mur 1.  
$Gconvext2$ est la convection sur la surface extérieure du mur 2.  $Gconvint2$ est la convection sur la surface intérieure du mur 2, ainsi que la convection sur les deux surfaces du mur intérieur.

In [ ]:
Gconvext1 = hconvext*Sm1
Gconvint1 = hconvint*Sm1
Gconvext2 = hconvext*Sm
Gconvint2 = hconvint*Sm 

#### Conduction #
$G_{conduction} = {{\lambda.S} \over {e}}$  
$Gcondm1$ est la conductivité de la moitié du mur 1, et $Gcondm2$ est la conductivité de la moitié du mur 2.

In [ ]:
Gcondm1 = lambois*Sm1*2/emext
Gcondm2 = lambois*Sm*2/emext

#### Controleur #
$G_{controleur} = K$

In [ ]:
Gc = 10**4

#### Conductance Totale du mur intérieur #

In [ ]:
Gint = lambois*Spf*2/ep + 1/((2/(Sm*hconvint))+lambois*Sm*2/emint)

#### Conductance de la porte et la fenêtre du mur 1

In [ ]:
G1 = Up*Spf+Uf*Spf

#### Conductance de la fenetre du mur 2

In [ ]:
G2 = Uf*Spf

### Calcul des Capacités Thermiques #
$C = m.C_{p}$  
$massem1$ est la masse du mur 1 et $massem2$ est la masse du mur 2. $C1$ est la capacité thermique du mur 1 et $C2$ est la capacité thermique du mur 2.

In [ ]:
massem1 = mv*Sm1*emext
massem2 = mv*Sm*emext
C1 = massem1*Csp
C2 = massem2*Csp

### Equations Différentielles Algébriques
$C.\dot{\theta}=-A^T.G.A\theta+A^T.G.b+f$  
Incidence matrix A

In [ ]:
A = np.zeros((13,8))
A[0,0]=1
A[1,0]=-1
A[1,1]=1
A[2,1]=-1
A[2,2]=1
A[3,2]=-1
A[3,3]=1
A[4,3]=1
A[5,3]=-1
A[5,4]=1
A[6,4]=1
A[6,5]=-1
A[7,5]=1
A[7,6]=-1
A[8,6]=1
A[8,7]=-1
A[9,7]=1
A[10,4]=1
A[11,3]=1
A[12,4]=1
print(A)

Conductance matrix G

In [ ]:
G=np.diag([Gconvext1,Gcondm1,Gcondm1,Gconvint1,G1,Gint,Gconvint2,Gcondm2,Gcondm2,Gconvext2,G2,Gc,Gc])
print(G)

Capacity matrix C

In [ ]:
C = np.zeros((8,8))
C[1,1]=C1
C[6,6]=C2
print(C)

Inputs - Temperature sources b

In [ ]:
B = np.zeros(13)
B[0]=Text
B[4]=Text
B[9]=Text
B[10]=Text
B[11]=Tc
B[12]=Tc
print(B)

Inputs - Flow rate sources f

In [ ]:
f = np.zeros(8)
f[0]=phi1
f[3]=Q1
f[4]=Q2
f[7]=phi2
print(f)

## PARTIE 2 - Régime Permanent #  
### Systemes d'Equations différentielles #
En régime permanent, on a $\dot{\theta}=0$, on résout alors $0=-A^T.G.A\theta+A^T.G.b+f$  


In [ ]:
AT = np.transpose(A)
INV = np.linalg.inv(np.dot(np.dot(AT,G),A))
ATGB = np.dot(np.dot(AT,G),B)
theta0 = np.dot(INV,ATGB+f)
print(theta0-273)

### Représentation d'état #
On utilise la représentation d'état, on souhaite extraire les valeurs de la température aux noeuds 3 et 4 (dans les deux pièces)

In [ ]:
Y = np.zeros(8)
Y[3]=1
Y[4]=1
[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, B, C, f, Y)

On résout alors :

In [ ]:
theta = -np.dot(np.dot(np.linalg.inv(As),Bs),U)
print(theta-273)

On remarque que les solutions obtenues avec le systeme différentiel et avec la représentation d'état sont les mêmes.

## PARTIE 3 - Modèle Dynamique : Réponse à un échelon #  
#### Détermination du pas de temps
Le pas de temps peut être obtenu avec $dt_{max}= min({{-2} \over {\lambda}})$

In [ ]:
eig = np.linalg.eig(As)[0]
dtmax = min(-2/eig) 
print(dtmax)

Le pas de temps obtenu est très grand, on choisit un pas de temps plus faible afin de mieux observer les phénomènes.

In [ ]:
dt = dtmax/10

#### Détermination du temps total

Le temps final correspond à 4 fois la constante de temps maximale. On determine ainsi le temps total de simulation. $n$ est le nombre de pas temporels, et $t$ le vecteur de temps.

In [ ]:
Ti=np.zeros(len(eig))
Ti=-1/eig
Tset = max(4*Ti)

duree = np.ceil(Tset / (3600)) * 3600
n = int(np.floor(duree / dt))    # number of time steps
t = np.arange(0, n * dt, dt)

#### Hypothèses de simulation
$u$ correspond aux entrées (Températures, Flux) sur la durée déterminée. $T_{implicite}$ et $T_{explicite}$ sont les températures choisies comme conditions initiales.

In [ ]:
u = np.zeros([len(U), n])
for k in range (len(U)):
    u[k,:]=U[k]*np.ones([1,n])

I = np.eye(2)
ns = As.shape[0]
Texplicit = np.zeros([ns, t.shape[0]]) 
Timplicit = np.zeros([ns, t.shape[0]])
Texplicit[:, 0]= [275]
Timplicit[:,0]=[275]

#### Résolution Numérique
On résout le système de représentation d'état avec les méthodes Euler implicites et explicites.  
Méthode d'Euler Explicite : $ \theta_{k+1} = (I + \Delta t A) \theta_{k} + \Delta t B u_k $
Méthode d'Euler Implicite : $\theta_{k+1} = (I - \Delta t A)^{-1} ( \theta_{k} + \Delta t B u_k )$

In [ ]:
for k in range(n - 1):
    Texplicit[:, k + 1] = (I + dt * As) @\
        Texplicit[:, k] + dt * Bs @ u[:, k]
    Timplicit[:, k + 1] = np.linalg.inv(I - dt * As) @\
        (Timplicit[:, k] + dt * Bs @ u[:, k])

yexplicit = Cs @ Texplicit + Ds @  u
yimplicit = Cs @ Timplicit + Ds @  u

On trace les résultats pour comparer

In [ ]:
fig, ax = plt.subplots()
ax.plot(t / 3600, yexplicit.T, t / 3600, yimplicit.T)
ax.set(xlabel='Time, $t$ / h',
       ylabel='Temperatue, $θ_i$ / °c',
       title='Step input: outdoor temperature $T_o$')
ax.legend(['Explicit', 'Implicit'])
ax.grid()
plt.show()

## Conclusion